In [1]:
# Импорт стандартных библиотек
import logging
import os
import uuid
from pathlib import Path

# Импорт библиотеки для загрузки переменных окружения из файла .env
from dotenv import load_dotenv

# Импорт аннотаций типов
from typing import Any, Dict, List

# Импорт внешних библиотек
import openai
from pydantic import BaseModel

# Импорт библиотек LangChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.schema import AgentAction, AgentFinish
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.docstore.document import Document

# Загрузка переменных окружения из файла .env
load_dotenv()

# настройка логгирования
logging.basicConfig(level=logging.INFO)

In [2]:
# Получение API ключей из переменной окружения
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
LANGCHAIN_TRACING_V2     = os.getenv('LANGCHAIN_TRACING_V2')
LANGCHAIN_API_KEY        = os.getenv('LANGCHAIN_API_KEY')
TAVILY_API_KEY           = os.getenv('TAVILY_API_KEY')
OPENAI_API_KEY           = os.environ['OPENAI_API_KEY']

In [3]:
openai.api_key = OPENAI_API_KEY

In [4]:
class FileManager:
    """
    Description:
        Класс для управления файлами, включая чтение, запись и добавление содержимого.
    """
    logging.basicConfig(level=logging.INFO)

    def __init__(self, working_directory: str = 'temp'):
        """
        Description:
            Инициализация рабочей директории.

        Args:
            working_directory: Путь к рабочей директории.
        """
        # Создаем рабочую директорию
        self.working_directory = Path(working_directory).absolute()
        logging.info("WORKING_DIRECTORY: %s", self.working_directory)

    @staticmethod
    def generate_run_id() -> str:
        """
        Description:
            Генерирует уникальный UUID.

        Returns:
            str: Сгенерированный UUID.
        """
        return str(uuid.uuid4())

    def read_document(self, file_name: str) -> str:
        """
        Description:
            Читает и возвращает содержимое файла.

        Args:
            file_name (str): Имя файла для чтения.

        Returns:
            str: Содержимое файла.
        """
        return FileManager._read_document(self.working_directory, file_name)

    @staticmethod
    def _read_document(working_directory: Path, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для чтения содержимого файла.

        Args:
            working_directory: Рабочая директория.
            file_name (str): Имя файла для чтения.

        Returns:
            str: Содержимое файла.
        """
        # Создаем путь к файлу с учетом рабочей директории
        file_path = working_directory / file_name.lstrip('/')
        logging.info(f"Attempting to read file from path: {file_path}")

        try:
            # Открываем файл для чтения
            with file_path.open("r", encoding='utf-8') as file:
                return file.read()
        except FileNotFoundError:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"File {file_name} not found at path: {file_path}")
            return f"File {file_name} not found."

    def write_document(self, content: str, file_name: str) -> str:
        """
        Description:
            Создает и сохраняет текстовый документ.

        Args:
            content: Текстовое содержимое для записи в файл.
            file_name: Имя файла для сохранения.

        Returns:
            str: Сообщение о сохранении файла.
        
        Raises:
            FileNotFoundError: Если файл не найден.
        """
        return FileManager._write_document(self.working_directory, content, file_name)

    @staticmethod
    def _write_document(working_directory: Path, content: str, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для записи документа.

        Args:
            working_directory: Рабочая директория.
            content: Текстовое содержимое для записи в файл.
            file_name: Имя файла для сохранения.

        Returns:
            str: Сообщение о сохранении файла.
        
        Raises:
            FileNotFoundError: Если файл не найден.
        """
        # Создаем путь к файлу и необходимые директории
        file_path = working_directory / file_name.lstrip('/')
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        try:
            # Открываем файл для записи
            with file_path.open("w", encoding='utf-8') as file:
                file.write(content)
            return f"Document saved to {file_name}"
        except IOError as e:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"Error writing to file {file_name}: {e}")
            return f"Error writing to file {file_name}: {e}"

    def append_document(self, content: str, file_name: str) -> str:
        """
        Description:
            Добавляет содержимое в конец существующего файла.

        Args:
            content: Текстовое содержимое для добавления в файл.
            file_name: Имя файла для добавления содержимого.

        Returns:
            str: Сообщение о сохранении файла.
        """
        return FileManager._append_document(self.working_directory, content, file_name)

    @staticmethod
    def _append_document(working_directory: Path, content: str, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для добавления содержимого в документ.

        Args:
            working_directory: Рабочая директория.
            content: Текстовое содержимое для добавления в файл.
            file_name: Имя файла для добавления содержимого.

        Returns:
            str: Сообщение о сохранении файла.
        """
        # Создаем путь к файлу и необходимые директории
        file_path = working_directory / file_name.lstrip('/')
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        try:
            # Открываем файл для добавления содержимого
            with file_path.open("a", encoding='utf-8') as file:
                file.write(content)
            return f"Document appended to {file_name}"
        except IOError as e:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"Error appending to file {file_name}: {e}")
            return f"Error appending to file {file_name}: {e}"

In [40]:
class AgentState():
    """
    Класс для представления состояния агента.
    """
    def __init__(self):
        self.messages: List[Dict[str, Any]] = []

class BaseAgent(AgentState):
    """
    Базовый класс для создания агентов.
    """

    def __init__(self, llm, system_prompt, tools: List[Any] = None):
        """
        Инициализация агента.

        Args:
            tools: Список инструментов, доступных агенту.
        """
        super().__init__()
        self.system_prompt = system_prompt
        self.llm = llm
        self.tools = tools or []

    def process_message(self, message: Dict[str, Any]) -> Dict[str, Any]:
        """
        Обрабатывает входящее сообщение и возвращает ответ.

        Args:
            message: Входящее сообщение.

        Returns:
            Ответ агента.
        """
        # Добавляем входящее сообщение в историю
        self.messages.append({"role": "user", "content": message["content"]})
        
        response = self.llm.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", 
                 "content": self.system_prompt},
                *self.messages
            ]
        ).choices

        # Добавляем ответ агента в историю
        self.messages.append({"role": "assistant", "content": response})

        return {"content": response}

    def call_tool(self, tool_name: str, **kwargs) -> Any:
        """
        Вызывает инструмент по имени.

        Args:
            tool_name: Имя инструмента.
            **kwargs: Дополнительные аргументы для инструмента.

        Returns:
            Результат работы инструмента.
        """
        for tool in self.tools:
            if tool.__class__.__name__ == tool_name:
                return tool.run(**kwargs)
        raise ValueError(f"Инструмент '{tool_name}' не найден.")

In [52]:
def pdf_loader(file_path: str) -> List[str]:
    """
    Description:
      Загружает PDF-файл и возвращает список документов.

    Args:
        file_path: Путь к PDF-файлу.

    Returns:
        Список документов, загруженных из PDF-файла.

    Raises:
        FileNotFoundError: Если указанный файл не найден.
        ValueError: Если файл не является допустимым PDF.

    Examples:
        >>> pdf_loader("example.pdf")
        ['Document content as a string.']
    """
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    
    return docs


In [58]:
# Загрузка PDF-файла
pdf_path = '/Users/cyberrunner/Downloads/Automatic Prompt Optimization with “Gradient Descent” and Beam Search.pdf'
pdf_pages = pdf_loader(pdf_path)

# Используем абсолютный путь к текущей директории
file_manager = FileManager(working_directory=os.getcwd())

# Инициализация агента
agent = BaseAgent(
    llm=openai,
    system_prompt=file_manager.read_document('prompts/system_prompt.txt'),
    tools=[]
)

# Итеративная суммаризация
summary = ""
for page in pdf_pages:
    summarized_content = agent.process_message(file_manager.read_document('prompts/chank_prompt.txt') + "\n" + page.page_content)
    summary += summarized_content + "\n"
    file_manager.append_document(summarized_content, 'summary.md')

# Запись окончательной суммаризации
file_manager.write_document(summary, 'final_summary.md')